# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

464


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Query 1 
### should display artist, song title and song's length that was heard during sessionId = 338, and itemInSession = 4 
 #### because we are working with NoSQL Database we have to do the following:
#### 1. logically construct the query that will give the  artist, song title and song's length based on sessionId, and itemInSession so it will be as follow 
 #### 2. write the query that creates the table that holds artist, song title and song's length based on sessionId, and itemInSession with primary key consists of sessionId, and itemInSession since they are our selecting criteria.
#### 3. after that, start inserting values into our table.
#### 4. now, it's.time for the select query.

## -----------------------------------------------------------------------------------------
##### Step 1 creat table songplay and set it prymary key to be (sessionId, itemInSession)

In [8]:

query = "CREATE TABLE IF NOT EXISTS songplay "
query = query + """(sessionid int , 
                    iteminsession int, 
                    artist text, 
                    song text, 
                    length float, 
                    PRIMARY KEY (sessionid, iteminsession))"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

##### Step 2 insert the values into our table 

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplay(sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query,(int(line[8]),int(line[3]),str(line[0]),str(line[9]),float(line[5])))

##### step3 Do a SELECT to verify that the data have been inserted into each table

In [11]:
query1 = "SELECT artist,song ,length FROM songplay WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query1)
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

## -----------------------------------------------------------------------------------------

## Query 2

### Here  we want to know name of the artist for and the song and the user name whos id = 10 and there sessionid = 182 .
#### 1. first creat thable logSongPlay that holdes userId ,sessionId,itemInSession,user_firstname ,user_lastname ,artist ,song_title with prymary key ((userId, sessionId), itemInSession)) 
#### 2. secound insert the valuse to our table 
#### 3. then, implement our selection statement.

In [12]:

query = "CREATE TABLE IF NOT EXISTS logsongplay "
query = query + """(userid int , 
                    sessionid int ,  
                    iteminsession int, 
                    firstname varchar, 
                    lastname varchar , 
                    artist text , 
                    song text, 
                    PRIMARY KEY ((userid, sessionid), iteminSession)) 
                    WITH CLUSTERING ORDER BY (iteminsession ASC);"""
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO logSongPlay (userid, sessionid, iteminsession, firstname, lastname , artist, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]) , int(line[3]), str(line[1]), str(line[4]), str(line[0]), str(line[9])))
        
query = "SELECT  artist, song, firstname, lastname FROM logsongplay WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname , row.userid, row.sessionid)

## -----------------------------------------------------------------------------------------

## Query 3

### Here  we want to know user name  who listened to the song 'All Hands Against His Own'
#### 1. first creat table userSongPlay that holdes userId ,sessionId,song_title, user_firstname ,user_lastname with PRIMARY KEY(userId, sessionId ,user_firstname))
#### 2. secound insert the valuse to our table 
#### 3. then, implement our selection statement.

In [13]:

query = "CREATE TABLE IF NOT EXISTS usersongplay "
query = query + """ (song text, 
                     userid int , 
                     firstname varchar, 
                     lastname varchar, 
                     PRIMARY KEY((song), userid))
                     WITH CLUSTERING ORDER BY (userid ASC);"""
                    
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO usersongplay ( song, userid, firstname, lastname )"
        query = query + " VALUES ( %s, %s, %s, %s)"
        session.execute(query,( str(line[9]), int(line[10]), str(line[1]), str(line[4])))
                        
query = "SELECT userid,firstname,lastname FROM usersongplay WHERE song='All Hands Against His Own'"                       
try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print( row.userid ,row.firstname, row.lastname)
    


### Drop the tables before closing out the sessions

In [14]:
query = "drop table IF EXISTS songplay"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS logsongplay"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table IF EXISTS usersongplay"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()